In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import random

In [7]:
# drive_path는 chromedriver 위치로 설정해주세요
# chromedriver는 chrome://settings/ 에서 버전을 확인해주시고, mac/linux/window os에 맞는 driver를 설치해주세요
# driver 위치 : https://chromedriver.chromium.org/downloads
class papago_rtt:
    def __init__(self, driver_path="C:/selenium/chromedriver.exe", src_lang='ko', tgt_lang='en'):

        self.driver_path = driver_path
        self.init_trans = f"https://papago.naver.com/?sk={src_lang}&tk={tgt_lang}"
        self.back_trans = f"https://papago.naver.com/?sk={tgt_lang}&tk={src_lang}"
        self.start_browser()
    
    def start_browser(self):
        self.driver = webdriver.Chrome(self.driver_path)
    
    def random_sleep(self):
        time.sleep(random.randint(10,30)/10)
        
    def __call__(self, query):
        #번역 페이지 열기
        self.driver.get(self.init_trans)
        self.random_sleep()

        #번역
        self.driver.find_element(By.CSS_SELECTOR, 'textarea#txtSource').send_keys(query)
        self.driver.find_element(By.CSS_SELECTOR, 'button#btnTranslate').click()
        self.random_sleep()

        # #번역 결과
        # output = self.driver.find_element(By.CSS_SELECTOR, 'div#txtTarget').text

        # #복원페이지 열기
        # self.driver.get(self.back_trans)
        # self.random_sleep()

        # #복원
        # self.driver.find_element(By.CSS_SELECTOR, 'textarea#txtSource').send_keys(output)
        # self.driver.find_element(By.CSS_SELECTOR, 'button#btnTranslate').click()
        # self.random_sleep()
        self.driver.find_element(By.CLASS_NAME, 'btn_switch___x4Tcl').click()
        self.random_sleep()
        #번역 결과
        output = self.driver.find_element(By.CSS_SELECTOR, 'div#txtTarget').text
        return output

In [8]:
import pandas as pd
train_data = pd.read_csv("./input/train_data.csv", index_col='index')
train_data.head()

,premise,hypothesis,label
index,,,
0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [14]:
rtt = papago_rtt()

index = []
premises = []
hypothesis = []
labels = []
exception_index = []

for i in range(len(train_data)):
    # try :
    # rtt_premise = rtt(train_data.loc[i, 'premise'])
    rtt_premise = rtt('')
    rtt_hypothesis = rtt(train_data.loc[i, 'hypothesis'])
    print(rtt_premise)
    if rtt_premise == '' or  rtt_hypothesis == '' :
        print('here')
    premises.append(rtt_premise)
    hypothesis.append(rtt_hypothesis)
    labels.append(train_data.loc[i, 'label'])
    index.append(i + len(train_data) + 1)
    if i % 50 == 0:
        print(f"steps : {i}")
    break
    # except :
    #     print(f'exception index : {i}')
    #     exception_index.append(i)
    
rtt_dataframe = pd.DataFrame({'index' : index, 'premise':premises, 'hypothesis' : hypothesis, 'label' : labels})
rtt_dataframe.to_csv('./input/train_rtt.csv', index=False)
    

<ipython-input-7-3d7fb80cfa2c>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.driver_path)


TypeError: __call__() missing 1 required positional argument: 'query'